# ClosenessCentrality
The _Closeness Centrality_ of a node measures the distance from that node to all other nodes.
Nodes with a high closeness score have the shortest distances to all other nodes.
The premise of this algorithm is that nodes with short distance to other nodes can spread information very efficiently through the network.
This is important for the availability of knowledge and resources.


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

In [3]:
create_graph_query = '''
CREATE (a:Node{id:"A"}),
       (b:Node{id:"B"}),
       (c:Node{id:"C"}),
       (d:Node{id:"D"}),
       (e:Node{id:"E"})
CREATE (a)-[:LINK]->(b),
       (b)-[:LINK]->(a),
       (b)-[:LINK]->(c),
       (c)-[:LINK]->(b),
       (c)-[:LINK]->(d),
       (d)-[:LINK]->(c),
       (d)-[:LINK]->(e),
       (e)-[:LINK]->(d);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 5, 'relationships-created': 8, 'nodes-created': 5, 'properties-set': 5}


In [4]:
streaming_query = """
CALL algo.closeness.stream('Node', 'LINKS') YIELD nodeId, centrality
RETURN nodeId,centrality order by centrality desc limit 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,nodeId,centrality
0,163,0.0
1,164,0.0
2,165,0.0
3,166,0.0
4,167,0.0
